# Trojans of Saturn

Set up using orbital elements as opposed to to xyz vectors. Start with Saturn's orbital elements, and change the a, ecc, and omega to create Trojans for Saturn. The trojans are 60 degrees ahead of Saturn, but we also want their perihelion to be 60 degrees  ahead of that of Saturn, so we only need to change omega. All hypothetical particles will have the same inc, Omega, and f as Saturn, their omega should be $\frac{\pi}{3}$ larger, and a and ecc would be taken from a grid. 
\
Let's run 20 particles per simulation, all with different semimajor axes and run six simulations, with particles in each having eccentricities between 0 and 0.1, with a 0.02 step.

## Build the sim

In [1]:
cd /Users/mariahjones/Desktop/Research/projects/Orbitals/

/Users/mariahjones/Desktop/Research/projects/Orbitals


In [1]:
import rebound
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
sim = rebound.Simulation()
sim.integrator = "trace"
#sim.dt = 1
#sim.save_to_file('trojans1.bin', interval=1e4,delete_file=True) # save as binary file

## Create hypothetical orbitals based on Saturn's orbital elements


The first particle has the same semi-major axis as Saturn, the last should be a one HiII radius larger orbit (fractional HiII radius is definted by Eq. 3.75 in Murray and Dermott). So, the largest initial semi-major axis should be a_saturn * (1 + alpha) where alpha is from Eq. 3.75. The picture on page 83 shows how the Trojan region has approximately this width, so that's why we want to have 20 particles distributed between a_saturn and a_saturn * (1 + alpha). Start with a single short simulation (~100 years or so) to test if it works.

In [4]:
# Convert au/days to au/years
vel_x = [0.0, -6.584186332911239E-03,1.361833139322700E-03, -3.176609453266956E-03, 1.106505958060795E-04]
vel_y = [0.0, 4.208054867943963E-03, 5.314892732297496E-03, 2.188750863994737E-03, 3.160539398425530E-03]
vel_z = [0.0, 1.298323287693746E-04, -1.466946539551684E-04, 4.926573452085599E-05, -6.757068282993264E-05]

vel_x = np.array(vel_x) * 365.25636
vel_y = np.array(vel_y) * 365.25636
vel_z = np.array(vel_z) * 365.25636


# Convert mass to integration units
sol_mass = (2 * np.pi)**2 # integration units
sol_gm = (1.33E20) # real units

jup_gm = 1.27E17 
jup_mass = (jup_gm / sol_gm) * sol_mass


sat_gm = 3.79E16
sat_mass = (sat_gm / sol_gm) * sol_mass

ur_gm = 5.79E15 
ur_mass = (ur_gm / sol_gm) * sol_mass

nep_gm = 6.84E15
nep_mass = (nep_gm / sol_gm) * sol_mass


# add particles to the simulation

sim.add(m = sol_mass, x = 0, y = 0, z = 0,
        vx = vel_x[0], vy = vel_y[0], vz = vel_z[0], hash = 'Sun')
sim.add(m = jup_mass, x = 2.559959519433896E+00, y = 4.317254861986015E+00, z =-7.520814690745442E-02,
        vx= vel_x[1], vy= vel_y[1], vz= vel_z[1], hash = 'Jupiter')
sim.add(m = sat_mass, x = 9.241056053800349E+00, y =-2.909785985421069E+00, z =-3.171865429820031E-01,
        vx= vel_x[2], vy= vel_y[2], vz= vel_z[2], hash = 'Saturn')
sim.add(m = ur_mass, x = 1.178746554190535E+01, y = 1.564471912991385E+01, z =-9.472686043780910E-02,
        vx= vel_x[3], vy= vel_y[3], vz= vel_z[3], hash = 'Uranus')
sim.add(m = nep_mass, x = 2.986403735565160E+01, y =-1.304275085444185E+00, z =-6.613439117521470E-01,
        vx= vel_x[4], vy= vel_y[4], vz= vel_z[4], hash = 'Neptune')

In [6]:
sim.status(showAllFields=False)

---------------------------------
REBOUND version:     	4.4.1
REBOUND built on:    	May  7 2024 19:50:41
Number of particles: 	5
Selected integrator: 	trace
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.particle.Particle object at 0x10f54a850, m=39.47841760435743 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.particle.Particle object at 0x10f54a7d0, m=0.03769743635904807 x=2.559959519433896 y=4.317254861986015 z=-0.07520814690745442 vx=-2.404915933520907 vy=1.5370188037454926 vz=0.04742208381662504>
<rebound.particle.Particle object at 0x10f54a850, m=0.01124986486620411 x=9.24105605380035 y=-2.909785985421069 z=-0.3171865429820031 vx=0.49741821539638226 vy=1.9412983731894378 vz=-0.053581155335124406>
<rebound.particle.Particle object at 0x10f54a7d0, m=0.0017186469017235304 x=11.78746554190535 y=15.64471912991385 z=-0.0947268604378091 vx=-1.1602768060418784 vy=0.7994551735295726 vz=0.0179946228638142>
<rebound.pa

In [5]:
ps = sim.particles
os = sim.orbits()


a_start = os[1].a
a_end = a_start * (1 + (sat_mass / 3 * sol_mass) ** 1/3)

sem_maj_ax = np.linspace(a_start, a_end, 20)
#print(sem_maj_ax)

eccs = [0.1, 0.08, 0.06, 0.04, 0.02, 0.]

ecc = eccs[0]
omega = os[1].omega + (np.pi / 3)
inc = os[1].inc
Omega = os[1].Omega
f = os[1].f

In [6]:
for i in range(20):
    a = sem_maj_ax[i]
    
    sim.add(a = a, e = ecc, inc = inc, omega = omega, Omega = Omega, f = f)

In [ ]:
sim.N_active = 5 # number of active particles, sun and four giants

In [7]:
Nout = 10  # number of points to display, corresponding to 10^4 years
tmax = 1000  # run the simulation for 10^8 years

In [8]:
#Nout = 10000  # number of points to display, corresponding to 10^4 years
#tmax = 100000000  # run the simulation for 10^8 years

times = np.linspace(0., tmax, Nout)

for time in times:
    sim.integrate(time)
    os = sim.orbits()  # Update os with the latest orbits
    remove_indices = [i for i, orbit in enumerate(os) if orbit.a <= 0]
    for i in sorted(remove_indices, reverse=True):
        sim.remove(i+1)